# 2.3 Izhikevich モデル
## 2.3.1 Izhikevich モデルの単純な実装
\textbf{Izhikevich モデル} (または\textbf{Simple model})は(Izhikevich, 2003)で考案されたモデルです. HHモデルのような生理学的な知見に基づいたモデルは実際のニューロンの発火特性をよく再現できますが、式が複雑化するため、数学的な解析が難しく、計算量が増えるために大規模なシミュレーションも困難となります。そこで、生理学的な正しさには目をつぶり、生体内でのニューロンの発火特性を再現するモデルが求められました。その特徴を持つのがIzhikevich モデルです(以下ではIzモデルと略します)。Izモデルは 2変数しかない\footnote{if文が入るためにシミュレーション上は簡易的ですが、解析をする上では難しくなります。(Bernardo, et al., 2008)       を読むといいそうです。}微分方程式ですが, 様々なニューロンの活動を模倣することができます.  
\begin{align}
C\frac{dv(t)}{dt}&=k\left(v(t)-v_r\right)\left(v(t)-v_t\right)-u(t)+I \\
\frac{du(t)}{dt}&=a\left\{b\left(v(t)-v_{r}\right)-u(t)\right\}
\end{align} 
ここで、$v$と$u$が変数であり, $v$は膜電位(membrane potential;単位はmV), $u$は回復電流(recovery current; 単位はpA)\footnote{ここでの「回復」というのは脱分極した後の膜電位が静止膜電位へと戻る、という意味です(対義語はactivationで膜電位の上昇を意味します)。
$u$は$v$の導関数において$v$の上昇を抑制するように$-u$で入っているため、$u$としてはK$^+$チャネル電流やNa$^+$チャネルの不活性化動態などが考えられます。}です。また、$C$は膜容量(membrane capacitance; 単位はpF), $v_r$は静止膜電位(resting membrane potential; 単位はmV), $v_t$は閾値電位(instantaneous threshold
potential; 単位はmV), $a$は回復時定数(recovery time constant; 単位はms$^{-1}$)の逆数(これが大きいと$u$が元に戻る時間が短くなります), 
$b$は$u$の$v$に対する感受性(共鳴度合い,  resonance; 単位はpA/mV)です。 $k$はニューロンのゲインに関わ定数で、小さいと発火しやすくなります(単位はpA/mV). \par
Izモデルの閾値の取り扱いはLIFモデルと異なり、HHモデルに近いです. LIFモデルでは閾値を超えた時に膜電位をピーク電位まで上昇させ(この過程は無くてもよいです)、続いて膜電位をリセットします。Izモデルの閾値は$v_t$ですが, 膜電位のリセットは閾値を超えたかで判断せず、膜電位$v$がピーク電位$v_{\text{peak}}$になったとき（または超えた時）に行います. そのためIzモデルの実際の閾値は膜電位の挙動が変化する(発火状態に移行する)、つまり分岐(bifurcation) が生じる点であり、パラメータの閾値$v_t$との間には差異があります。\par
さて、膜電位がピーク電位$v_{\text{peak}}$に達したとき(すなわちif $v \geq v_{\text{peak}}$)、$u, v$を次のようにリセットします\footnote{バースト発火(bursting)の挙動を表現するためには、速い回復変数(fast recovery variable)と遅い回復変数(slow recovery variable)の2つが必要となります(従って膜電位も合わせて全部で3変数必要)。一方で、IzモデルではLIFモデルのようなif文によるリセットを用いているため、速い回復変数が必要なく、遅い回復変数$u$のみでバースト発火を表現できます。}。
\begin{align} 
u&\leftarrow u+d\\
v&\leftarrow v_{\text{reset}}
\end{align}
とします. ただし, $v_{\text{reset}}$は過分極を考慮して静止膜電位$v_r$よりも小さい値とします. また、$d$はスパイク発火中に活性化される正味の外向き電流の合計を表し、発火後の膜電位の挙動に影響します(単位はpA)。\par
以上を踏まえて, シミュレーションの例と実装は次のようになります\footnote{コードは\texttt{/SingleFileSimulations/Neurons/Izhikevich\_single.py}です。}. 
ここで注意してほしいのが、時間のスケールです。LIFでは秒(sec)のスケールでしたが、Izモデルでは更新においてミリ秒のスケールを用います(あくまでこの本では、です)。

## 2.3.2 様々な発火パターンのシミュレーション
次に様々な発火パターンを模倣するようにIzモデルの定数を変化させてみましょう。Intrinsically Bursting (IB)ニューロンとChattering (CH) ニューロン(または fast rhythmic bursting (FRB) ニューロン)のシミュレーションを行います。基本的には定数を変えるだけです。

結果は\ref{fig:Iz_2}のようになります。